In [1]:
import pandas as pd

pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 999)
pd.set_option('display.max_rows', 999)
pd.set_option('display.min_rows', 999)

%config IPCompleter.use_jedi = False

In [2]:
df = pd.read_csv('../data/df_clean_unreg.csv')
df.head()

,Unnamed: 0,num_asian,num_black,num_hisp,num_multi
0,0,269.0,5952.0,918.0,0.0
1,1,360.0,5281.0,940.0,77.0
2,2,360.0,5281.0,940.0,77.0
3,3,360.0,5281.0,940.0,77.0
4,4,382.0,4894.0,908.0,96.0
